# Read Data

In [9]:
import cupy as cp

ImportError: No module named cupy

In [1]:
import csv
import pandas
import numpy
if __name__ == "__main__":
    dfApp = pandas.read_csv('data/app_dataframe_new.csv')
    dfArt = pandas.read_csv('data/article.csv')

    article = list(dfArt)
    article = article[1:]
    category = list(dfApp)
    category = category[3:]
    user = dfApp.user_code.tolist()
    uid = dfApp.user_id.tolist()    
    
    #delete the column names and userids/usercodes, only keep the freq values
    Uapp = dfApp.values
    Uapp = numpy.delete(Uapp, numpy.s_[0:3], 1)
    
    #delete the column names and userids/usercodes, only keep the dwelltime values
    dfArt.dropna(axis=1, how='all')
    Uart = dfArt.values
    Uart = numpy.delete(Uart, 0, 1)
    
    #to speed up, only use the top-20 to predict
    user = user[:20]
    uid = uid[:20]
    Uapp = Uapp[:20]
    Uart = Uart[:20]
    
    print Uapp,Uart


[[nan nan nan ..., 1.0 1.0 nan]
 [nan nan nan ..., nan nan nan]
 [nan nan nan ..., nan nan nan]
 ..., 
 [nan nan nan ..., 1.0 2.0 1.0]
 [nan nan nan ..., 3.0 1.0 nan]
 [nan nan nan ..., nan 2.0 nan]] [[ nan  nan  nan ...,  nan  nan  nan]
 [ nan  nan  nan ...,  nan  nan  nan]
 [ nan  nan  nan ...,  nan  nan  nan]
 ..., 
 [ nan  nan  nan ...,  nan  nan  nan]
 [ nan  nan  nan ...,  nan  nan  nan]
 [ nan  nan  nan ...,  nan  nan  nan]]


In [3]:
print len(Uart[0])

54071


In [7]:
#delete articles with all nan values
dfArtP = dfArt.dropna(axis=1, how='all')
articleP = list(dfArtP)
Uartp = dfArtP.values
Uartp = numpy.delete(Uartp, 0, 1)
Uartp = Uartp[:20]


# Proposed Model: Collective Matrix Factorization

In [2]:
# -*- coding: utf-8 -*-
"""
Created on Tue Nov 14 16:10:11 2017
@author: Mika
"""

import numpy.linalg as LA
###############################################################################

"""
@INPUT:
    R     : a matrix to be factorized, dimension N x M
    I     : a matrix to be added,dimension N x C
    P     : an initial matrix of dimension N x K
    Q     : an initial matrix of dimension M x K
    S     : an initial matrix of dimension C x K
    K   : the number of latent features for R and I
    R=P * Q;
    I=P * S；
    steps : the maximum number of steps to perform the optimisation
    alpha : the learning rate
    beta  : the regularization parameter
@OUTPUT:
    the final matrices P and Q and S
"""    
def collective_matrix_factorization(R, I, P, Q, S, K, steps=5000, lamda=0.6, alpha=0.0002, beta=0.02):
    Q = Q.T
    if S!=None:
        S = S.T        
    for step in xrange(steps):
        for i in xrange(len(R)):
            for j in xrange(len(R[i])):
                if R[i][j] > 0:
                    eij = R[i][j] - numpy.dot(P[i,:],Q[:,j])
                    if I!=None:#have transfer learning#
                        for x in xrange(len(I[i])):
                            if I[i][x] >0:
                                eix= I[i][x] - numpy.dot(P[i,:],S[:,x])
                                for k in xrange(K):
                                    P[i][k] = P[i][k] + alpha * (2 * lamda * eij * Q[k][j] + 2 * (1 - lamda) * eix * S[k][x] - beta * P[i][k])
                                    Q[k][j] = Q[k][j] + alpha * (2 * lamda * eij * P[i][k] - beta * Q[k][j])
                                    S[k][x] = S[k][x] + alpha * (2 * (1-lamda) * eix * P[i][k] - beta * S[k][x])          
                    else:#no transfer learning#
                        for k in xrange(K):
                            P[i][k] = P[i][k] + alpha * (2 * eij * Q[k][j] - beta * P[i][k])
                            Q[k][j] = Q[k][j] + alpha * (2 * eij * P[i][k] - beta * Q[k][j])
        e_1=0
        e_2=0
        reg=0 
        for i in xrange(len(R)):
            for j in xrange(len(R[i])):
                if R[i][j] > 0:
                    e_1 = e_1 + pow(R[i][j] - numpy.dot(P[i,:],Q[:,j]), 2)
                    for k in xrange(K):
                        reg = reg + (beta/2) * ( pow(P[i][k],2) + pow(Q[k][j],2) )
            
        if I !=None:
            for x in xrange(len(I[i])):
                if I[i][x] > 0:
                    e_2 = e_2 + pow(I[i][x] - numpy.dot(P[i,:],S[:,x]), 2)
                    for k in xrange(K):
                        reg = reg + (beta/2) * ( pow(S[k][x],2) )  
        
            e=lamda*e_1+(1-lamda)*e_2+reg
        else:
            e=e_1+reg
        if e < 0.001:
            break
    print ('the loss is:')
    print e
    return P, Q.T

def RMSE(X, Y):
    S=pow(X-Y,2)
    S=S.mean()
    return numpy.sqrt(S.mean)
###############################################################################

if __name__ == "__main__":
    
    N3 = len(Uartp)#the row of R#

    M3 = len(Uartp[1])#the colume of user*article#
#    print M
    K = 5
    C = len(Uapp[1])#the colume of user*apps#

    P = numpy.random.rand(N,K)
    Q = numpy.random.rand(M,K)
    S = numpy.random.rand(C,K)
    nP, nQ = collective_matrix_factorization(Uartp,Uapp, P, Q, S, K)

    nR=numpy.dot(nP,nQ.T)
    print nR

/home/yukic/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:28: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
/home/yukic/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:35: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
/home/yukic/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:57: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.


the loss is:
240.940765324
[[ 10.92175307   9.42674427  12.04506084 ...,  10.48764618  14.05591117
   11.97393651]
 [  8.52127704   7.9174917   11.9840604  ...,  11.89235617  12.42876354
   10.67145536]
 [ 11.1471215   10.46084603  15.58752386 ...,  15.42244032  16.32686009
   13.60560391]
 ..., 
 [  0.73930666   0.6191654    0.80275844 ...,   1.07204748   1.69398424
    1.14633165]
 [  6.51509669   7.08241845   9.72195364 ...,   8.8728669   10.25361635
    9.05135171]
 [  5.12535264   6.01674791   8.82800992 ...,   9.79184033  10.58502213
    6.69879962]]


In [10]:
    N3 = len(Uartp)#the row of R#

    M3 = len(Uartp[1])#the colume of user*article#

    P3 = numpy.random.rand(N3,K)
    Q3 = numpy.random.rand(M3,K)
    nP3, nQ3 = collective_matrix_factorization(Uartp,Uapp, P3, Q3, S, K)

    nR3=numpy.dot(nP3,nQ3.T)
    print nR3

/home/yukic/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:28: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
/home/yukic/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:35: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
/home/yukic/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:57: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.


KeyboardInterrupt: 

# Baseline 1 - Matrix Factorization

In [5]:
#######################################
#  Baseline 1 - Matrix Factorization  #
#######################################
#!/usr/bin/python
#
# Created by Albert Au Yeung (2010)
#
# An implementation of matrix factorization
#

###############################################################################

"""
@INPUT:
    R     : a matrix to be factorized, dimension N x M
    P     : an initial matrix of dimension N x K
    Q     : an initial matrix of dimension M x K
    K     : the number of latent features
    steps : the maximum number of steps to perform the optimisation
    alpha : the learning rate
    beta  : the regularization parameter
@OUTPUT:
    the final matrices P and Q
"""
def matrix_factorization(R, P, Q, K, steps=5000, alpha=0.0002, beta=0.02):
    Q = Q.T
    for step in xrange(steps):
        for i in xrange(len(R)):
            for j in xrange(len(R[i])):
                if R[i][j] > 0:
                    eij = R[i][j] - numpy.dot(P[i,:],Q[:,j])
                    for k in xrange(K):
                        P[i][k] = P[i][k] + alpha * (2 * eij * Q[k][j] - beta * P[i][k])
                        Q[k][j] = Q[k][j] + alpha * (2 * eij * P[i][k] - beta * Q[k][j])
        eR = numpy.dot(P,Q)
        e = 0
        for i in xrange(len(R)):
            for j in xrange(len(R[i])):
                if R[i][j] > 0:
                    e = e + pow(R[i][j] - numpy.dot(P[i,:],Q[:,j]), 2)
                    for k in xrange(K):
                        e = e + (beta/2) * ( pow(P[i][k],2) + pow(Q[k][j],2) )
        if e < 0.001:
            break
    return P, Q.T

###############################################################################

## Prediction 

In [7]:
N1 = len(Uart)
M1 = len(Uart[0])
K1 = 10
P1 = numpy.random.rand(N1,K1)
Q1 = numpy.random.rand(M1,K1)
nP1, nQ1 = matrix_factorization(Uart, P1, Q1, K1)
nR1 = numpy.dot(nP, nQ.T)
print nR1

NameError: name 'nP' is not defined

In [10]:
nR1 = numpy.dot(nP1, nQ1.T)
print nR1
print len(nR1[0])

[[ 13.22129478  15.69279804  20.60779168 ...,  19.20493175  15.37956037
   14.79524474]
 [  9.81908417  13.35799222  15.63813441 ...,  15.02674457  12.82553556
   14.17261944]
 [ 13.74309385  20.5247801   21.02819248 ...,  19.74240135  16.60512615
   17.1042785 ]
 ..., 
 [  1.79125159   2.35851272   1.93035058 ...,   2.39469861   2.18021667
    2.25291844]
 [  9.90894749  12.95289356  14.52034103 ...,  13.88367714  11.93274785
   13.09739762]
 [  7.93358586  12.34800328  13.96431173 ...,  13.93191535  11.45911791
   10.95857606]]
54071


## Final Result

In [13]:
topR = []
for i in range(nR1.shape[0]):
    lineR = []
    #return the index with ranked 1~5
    subRank = nR1[i].argsort()[-5:][::-1]
    print 'user #',user[i],uid[i]
#     print subRank
    for j in range(5):
        tmp = article[subRank[j]]
        lineR.append(tmp)
    print lineR
    
# print topR

user # 0 00108FC5F1F3031ADD536548124A4912
['25942', '27436', '44347', '53592', '22344']
user # 1 00310455B7B24D5CDEC892351B8D1D51
['25942', '27436', '44347', '53592', '25002']
user # 2 00381B68ACD6CC83635CDDD9B40F0BAF
['25942', '53592', '44347', '27436', '25002']
user # 3 003B2215B070CDB50C0819C72E208F0A
['25942', '44347', '21097', '53592', '27436']
user # 4 003B8D414199B9C5EB7CF8C94BCC6787
['25942', '27436', '44347', '25002', '22344']
user # 5 0043B1ADB92D7265FCF0B744A42AAF05
['25942', '44347', '27436', '53592', '34968']
user # 6 00450B9C8D0055C699C03D6A0B1F6CE8
['44347', '27436', '25942', '25002', '53592']
user # 7 0049D0D7726B7A1DEFB26EFE6B336B38
['44347', '25942', '27436', '25002', '44684']
user # 8 004A20E76B5AFEBF06EC46D0FD33A404
['25942', '44347', '27436', '53592', '25002']
user # 9 004A5982BCE2DECD0622672F6DFE488E
['25942', '53592', '44347', '27436', '25002']
user # 10 0064E21780EE64B8C509AB0AA34F5EE6
['53592', '25942', '27436', '44347', '34968']
user # 11 00655FA53221463FF6900

In [5]:
import math
topR1 = []
for i in range(nR.shape[0]):
    lineR1 = []
    lineR2 = []
    #return the index with ranked 1~5
    subRank = nR[i].argsort()[-5:][::-1]
    subRank2 = Uart[i].argsort()
    print 'user #',user[i],uid[i]
#     print subRank
    for j in range(5):
        tmp = article[subRank[j]]
        tmp2 = article[subRank2[j]]
        lineR1.append(tmp)
    for k in range(nR.shape[0]):
        if math.isnan(Uart[i][subRank2[k]]):
            continue
        tmp2 = article[subRank2[k]]
        lineR2.append(tmp2)
        if len(lineR2)==5:
            break
    print lineR1
    print lineR2
    
# print topR

user # 0 00108FC5F1F3031ADD536548124A4912
['44347', '27436', '25002', '25942', '44684']
['22344']
user # 1 00310455B7B24D5CDEC892351B8D1D51
['44347', '27436', '25942', '25002', '44684']
['40351']
user # 2 00381B68ACD6CC83635CDDD9B40F0BAF
['44347', '27436', '25942', '25002', '44684']
['12897', '16891', '35968', '19071', '40613']
user # 3 003B2215B070CDB50C0819C72E208F0A
['25942', '44347', '27436', '53592', '25002']
['41089', '4367', '8597', '4283', '13976']
user # 4 003B8D414199B9C5EB7CF8C94BCC6787
['25942', '44347', '27436', '25002', '53592']
[]
user # 5 0043B1ADB92D7265FCF0B744A42AAF05
['44347', '27436', '25942', '25002', '44684']
['34968']
user # 6 00450B9C8D0055C699C03D6A0B1F6CE8
['44347', '27436', '25942', '25002', '44684']
[]
user # 7 0049D0D7726B7A1DEFB26EFE6B336B38
['44347', '27436', '25002', '25942', '44684']
[]
user # 8 004A20E76B5AFEBF06EC46D0FD33A404
['44347', '25942', '27436', '25002', '44684']
['46595', '22512', '5905', '34612', '37315']
user # 9 004A5982BCE2DECD0622672F6D

In [21]:
# topR1 = []
for i in range(nR.shape[0]):
    lineR2 = []
    #return the index with ranked 1~5
    subRank2 = Uart[i].argsort()
    print 'user #',user[i],uid[i]
    for j in range(nR.shape[0]):
        if math.isnan(Uart[i][subRank2[j]]):
            continue
        tmp2 = article[subRank2[j]]
        lineR2.append(tmp2)
        if len(lineR2)==5:
            break
    print lineR2
    
# print topR

user # 0 00108FC5F1F3031ADD536548124A4912
['22344']
user # 1 00310455B7B24D5CDEC892351B8D1D51
['40351']
user # 2 00381B68ACD6CC83635CDDD9B40F0BAF
['12897', '16891', '35968', '19071', '40613']
user # 3 003B2215B070CDB50C0819C72E208F0A
['41089', '4367', '8597', '4283', '13976']
user # 4 003B8D414199B9C5EB7CF8C94BCC6787
[]
user # 5 0043B1ADB92D7265FCF0B744A42AAF05
['34968']
user # 6 00450B9C8D0055C699C03D6A0B1F6CE8
[]
user # 7 0049D0D7726B7A1DEFB26EFE6B336B38
[]
user # 8 004A20E76B5AFEBF06EC46D0FD33A404
['46595', '22512', '5905', '34612', '37315']
user # 9 004A5982BCE2DECD0622672F6DFE488E
[]
user # 10 0064E21780EE64B8C509AB0AA34F5EE6
['19664']
user # 11 00655FA53221463FF6900791BB1B55C4
['25162']
user # 12 006C8B449F56C9F3B779CBBA8AEE740B
['51694']
user # 13 00889B8C61B6549DE1A17EF80DBCE9A4
['12996', '19516', '7158', '33971', '5876']
user # 14 008F468ED81D8CEE56D77934C2E2DA6C
['52028', '27683', '40479', '50596', '43185']
user # 15 009D8C610CBB6526C22EC43721A53B9C
['35852']
user # 16 00A606

In [17]:
print dfArt.44347

SyntaxError: invalid syntax (<ipython-input-17-36bb4c8c0191>, line 1)

# Baseline 2 - ?

In [3]:
#8*5 empty
w, h = 8, 5;
Matrix = [[None for x in range(w)] for y in range(h)] 
print Matrix

[[None, None, None, None, None, None, None, None], [None, None, None, None, None, None, None, None], [None, None, None, None, None, None, None, None], [None, None, None, None, None, None, None, None], [None, None, None, None, None, None, None, None]]
